In [1]:
### Running this notebook to generate the model artifacts and the run ids on the server 

In [2]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
from sklearn.pipeline import make_pipeline

In [4]:
import mlflow

mlflow.set_tracking_uri("http://3.253.72.252:5000")
mlflow.set_experiment("green-taxi-duration_rf")

<Experiment: artifact_location='s3://mlflow-abiodun/4', creation_time=1723675293405, experiment_id='4', last_update_time=1723675293405, lifecycle_stage='active', name='green-taxi-duration_rf', tags={}>

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [8]:
# need to download files and add them to the folder
df_train = read_dataframe('/workspaces/mlops-zoomcamp/04-deployment/web-service-mlflow/data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('/workspaces/mlops-zoomcamp/04-deployment/web-service-mlflow/data/green_tripdata_2022-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [12]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

    # with open('dict_vectorizer' , 'wb') as f_out:
    #     pickle.dump(dv , f_out)

    # mlflow.log_artifact('dict_vectorizer.bin')

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.101201727016494


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024/08/15 06:07:21 WARNING mlflow.models.model: Input example should be provided to infer 

In [ ]:
# Downloading the model artifacts from the registry

In [8]:
from mlflow.tracking import MlflowClient


In [10]:
MLFLOW_TRACKING_URI = 'http://34.246.199.26:5000'
RUN_ID = '004d5b39c44d4a37b0ca53421544dcc5'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [13]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [14]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

IsADirectoryError: [Errno 21] Is a directory: '/tmp/tmpbdm0hjg1/model'

In [23]:
dv

DictVectorizer()